In [10]:
def parse_sid(sid_text):
    print("inside parse_sid")
    pattern = re.compile(r"\d+\.\s*(.+?)\s*->\s*(.+?):\s*(.+)")
    sid = []
    for line in sid_text.splitlines():
        match = pattern.match(line.strip())
        if match:
            sender = match.group(1).strip()
            receiver = match.group(2).strip()
            message = match.group(3).strip()
            sid.append((f"{sender} -> {receiver}", message))
    return sid

In [11]:
##############SID Graph##########################
import os


from graphviz import Digraph
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_md")  # Make sure this is installed

# Define canonical entities
canonical_entities = [
    "Customer",
    "E-commerce Website"    
]

# Convert to spaCy Docs for similarity comparison
canonical_docs = {ent: nlp(ent) for ent in canonical_entities}

def normalize_entity(entity):
    entity_doc = nlp(entity)
    best_match = None
    best_score = 0.75  # Similarity threshold (tune as needed)

    for canon_name, canon_doc in canonical_docs.items():
        score = entity_doc.similarity(canon_doc)
        if score > best_score:
            best_match = canon_name
            best_score = score

    return best_match if best_match else entity.strip()

def draw_sid_graph(sid_list, output_path="sid_graph"):
    dot = Digraph(comment="SID - Sequence Interaction Diagram")
    dot.attr(dpi='300')  # increase DPI for sharper text
    dot.attr(fontsize='14')  
    dot.attr(rankdir='LR',splines='polyline')  # Left to right direction
       
    # Deduplicate using a set
    normalized_edges = set()

    for interaction, action in sid_list:
        source, target = [normalize_entity(s.strip()) for s in interaction.split("->")]
        key = (source, target, action.strip())

        if key in normalized_edges:
            print(f"Skipping duplicate: {source} -> {target} [{action}]")
            continue

        normalized_edges.add(key)
        dot.node(source, source, shape="box", style="filled", fillcolor="lightblue")
        dot.node(target, target, shape="box", style="filled", fillcolor="lightgreen")
        dot.edge(source, target, label=action)

    # Render graph to file
    dot.render(output_path, format='png', cleanup=True)
    print(f"SID graph saved to: {output_path}.png")

In [12]:
def extract_sbd_section(full_text):
    lines = full_text.splitlines()
    sbd_start = None
    for i, line in enumerate(lines):
         if "SBD" in line.upper():
            sbd_start = i + 1  # start after this line
            break
    if sbd_start is None:
        print("No SBD section found!")
        return ""

    # Extract until explanation or EOF
    sbd_lines = []
    for line in lines[sbd_start:]:
        if line.strip().startswith("### Explanation"):
            break
        sbd_lines.append(line)

    return "\n".join(sbd_lines).strip()

In [13]:
from graphviz import Digraph
import re
import os

import re

def parse_sbd_(sbd_text):
    """
    Parses the SBD text into a dict of {subject_name: sbd_text_for_that_subject}
    Assumes subjects are marked by lines like: '#### Subject Name:'
    Returns: dict of subject -> string (SBD text)
    """
    subjects = {}
    current_subject = None
    current_lines = []

    for line in sbd_text.splitlines():
       
        subject_match = re.match(r'^\s*####\s*(.+?):\s*$', line)
        if subject_match:
            # Save the previous subject's lines
            if current_subject is not None and current_lines:
                subjects[current_subject] = "\n".join(current_lines).strip()

            # Start a new subject
            current_subject = subject_match.group(1).strip()
            current_lines = []
        else:
            if current_subject is not None:
                current_lines.append(line)

    # Save the last subject block
    if current_subject is not None and current_lines:
        subjects[current_subject] = "\n".join(current_lines).strip()

    return subjects

In [14]:
from graphviz import Digraph
import re

def draw_sbd_graph(sbd_text, output_path):
    """
    Parse SBD text and create a Graphviz SBD diagram.
    Saves as PDF at `output_path`.
    """
    dot = Digraph("SBD", format='pdf')  
    dot.attr(rankdir='LR')
    
    def add_edge_safe(src, dst, label=""):
        if dst is not None and dst != src:
            dot.edge(str(src), str(dst), label=label)

    state_styles = {
        'StartState': {"shape": "rectangle", "color": "yellow"},
        'EndState':   {"shape": "rectangle", "color": "yellow"},
        'SendState':  {"shape": "rectangle", "color": "green"},
        'ReceiveState': {"shape": "rectangle", "color": "pink"},
        'DoState':    {"shape": "rectangle", "color": "yellow"},
        'Unknown':    {"shape": "rectangle", "color": "lightgrey"},
    }

    steps = {}
    lines = sbd_text.strip().splitlines()
    current_step = None
    i = 0

    while i < len(lines):
        line = lines[i].strip()
        match = re.match(r'^(\d+)\.\s*\*{0,2}(\w+)\*{0,2}:\s*(.*)', line)
        if match:
            num, state_type, label = match.groups()
            current_step = int(num)

            if state_type == "GotoStep":
                target_step = int(label.strip().split()[0])
                steps[current_step] = {
                    "type": "GotoStep",
                    "label": "",
                    "choices": [],
                    "branches": [],
                    "goto": target_step
                }
            else:
                steps[current_step] = {
                    "type": state_type,
                    "label": label,
                    "choices": [],
                    "branches": [],
                    "goto": None
                }

                # Lookahead for extra details
                if state_type == "SendState":
                    if i + 1 < len(lines) and lines[i + 1].strip().startswith("To:"):
                        steps[current_step]["To"] = lines[i + 1].strip().split(":", 1)[1].strip()
                        i += 1
                    if i + 1 < len(lines) and lines[i + 1].strip().startswith("Msg:"):
                        steps[current_step]["Msg"] = lines[i + 1].strip().split(":", 1)[1].strip()
                        i += 1
                    if i + 1 < len(lines) and lines[i + 1].strip().startswith("Next:"):
                        steps[current_step]["next"] = int(lines[i + 1].strip().split(":", 1)[1].strip())
                        i += 1

                if state_type in {"StartState", "DoState"}:
                    if i + 1 < len(lines) and lines[i + 1].strip().startswith("OutgoingLabel:"):
                        steps[current_step]["OutgoingLabel"] = lines[i + 1].strip().split(":", 1)[1].strip()
                        i += 1
                    elif i + 1 < len(lines) and lines[i + 1].strip().startswith("Description:"):
                        steps[current_step]["OutgoingLabel"] = lines[i + 1].strip().split(":", 1)[1].strip()
                        i += 1
               
                # --- HANDLE ReceiveState (multi-line lookahead) ---
                if state_type == "ReceiveState":
                    # scan subsequent lines until hit a new numbered step or an empty line
                    j = i + 1
                    while j < len(lines):
                        nxt = lines[j].strip()
                        # stop if next line is the start of a new numbered state
                        if re.match(r'^\d+\.\s*\*{0,2}\w+\*{0,2}:\s*', nxt):
                            break

                        # '- From:' or 'From:' (support both bullet and no-bullet formats)
                        if nxt.startswith('- From:') or nxt.startswith('From:'):
                            from_actor = nxt.split(':', 1)[1].strip()
                            msg = ""
                            next_step = None
                            # look for Msg: and Next: in following lines
                            if j + 1 < len(lines) and lines[j + 1].strip().startswith('Msg:'):
                                msg = lines[j + 1].strip().split(':', 1)[1].strip()
                                j += 1
                            if j + 1 < len(lines) and lines[j + 1].strip().startswith('Next:'):
                                next_step = int(lines[j + 1].strip().split(':', 1)[1].strip())
                                j += 1
                            steps[current_step]["choices"].append({
                                "from": from_actor,
                                "msg": msg,
                                "next": next_step
                            })
                        # case: 'Msg:' then 'Next:' (no From)
                        elif nxt.startswith('Msg:'):
                            msg = nxt.split(':', 1)[1].strip()
                            next_step = None
                            if j + 1 < len(lines) and lines[j + 1].strip().startswith('Next:'):
                                next_step = int(lines[j + 1].strip().split(':', 1)[1].strip())
                                j += 1
                            steps[current_step]["choices"].append({
                                "from": "",
                                "msg": msg,
                                "next": next_step
                            })
                        # case: direct 'Next:' (no From/Msg)
                        elif nxt.startswith('Next:'):
                            next_step = int(nxt.split(':', 1)[1].strip())
                            steps[current_step]["choices"].append({
                                "from": "",
                                "msg": "",
                                "next": next_step
                            })
                        # other lines: skip (or break on blank line)
                        elif nxt == "" :
                            break
                        j += 1

                    # advance i so the main loop continues after the consumed lookahead lines
                    i = j - 1

            i += 1
            continue
            
           

        
          
        
        
        # Parse DoState branches
        elif line.startswith('- Step:') and current_step is not None:
            step = int(line.split(':')[1].strip())
            desc = ""
            if i + 1 < len(lines) and lines[i + 1].strip().startswith('Description:'):
                desc = lines[i + 1].strip().split(':', 1)[1].strip()
                i += 1

            steps[current_step]["branches"].append({
                "step": step,
                "desc": desc
            })

        elif 'GotoStep' in line and current_step is not None:
            match = re.match(r'\*{0,2}GotoStep\*{0,2}:\s*(\d+)', line)
            if match:
                steps[current_step]["goto"] = int(match.group(1))

        i += 1

    # Add missing goto targets
    goto_targets = {info["goto"] for info in steps.values() if info["goto"] is not None}
    for tgt in goto_targets:
        if tgt not in steps:
            steps[tgt] = {
                "type": "Unknown",
                "label": f"Step {tgt} \\nundefined",
                "choices": [],
                "branches": [],
                "goto": None
            }

    
    # Find all reachable steps
    reachable = set()
    for num, info in steps.items():
        # Direct next
        if "next" in info and info["next"] is not None:
            reachable.add(info["next"])
        # ReceiveState choices
        for c in info.get("choices", []):
            if c.get("next") is not None:
                reachable.add(c["next"])
        # DoState branches
        for b in info.get("branches", []):
            reachable.add(b["step"])
        # GotoStep
        if info.get("goto") is not None:
            reachable.add(info["goto"])
        
    #include sequential fallback targets
    all_step_nums = sorted(steps.keys())
    for idx, num in enumerate(all_step_nums):
        info = steps[num]
        next_step = all_step_nums[idx + 1] if idx + 1 < len(all_step_nums) else None
        uses_fallback = False
        if info["type"] == "StartState":
            uses_fallback = True
        elif info["type"] == "SendState" and info.get("next") is None:
            uses_fallback = True
        elif info["type"] == "DoState" and not info.get("branches"):
            uses_fallback = True
        if uses_fallback and next_step is not None:
            reachable.add(next_step)

    # Draw only reachable + start nodes
    for num, info in steps.items():
        if num != min(steps.keys()) and num not in reachable:
            continue  # skip orphans except the start step

        if info["type"] == "GotoStep":
            continue
        if info["type"] == "Unknown" and info["label"].endswith("undefined"):
            continue
            
      

        style = state_styles.get(info["type"], {"shape": "rectangle", "color": "lightgrey"})
       
        dot.node(str(num), label=info["label"], shape=style["shape"], style="filled", fillcolor=style["color"])

    # Draw edges
    all_step_nums = sorted(steps.keys())
    for idx, num in enumerate(all_step_nums):
        info = steps[num]

        if info["type"] == "GotoStep":
            continue
        if info["type"] == "EndState":
            continue

        
        if info["type"] == "ReceiveState":
            if info["choices"]:
                # Only draw from choices
                for choice in info["choices"]:
                    label = f"From: {choice['from']}\\nMsg: {choice['msg']}"
                    if choice["next"] is not None:
                        
                        add_edge_safe(num, choice["next"], label)
            else:
                # No choices, fall back to single Next
                if "next" in info and info["next"] is not None:
                    
                    add_edge_safe(num, info["next"], "")
           

        elif info["type"] == "SendState":
            label_parts = []
            if "To" in info:
                label_parts.append(f"To:{info['To']}")
            if "Msg" in info:
                label_parts.append(f"Msg:{info['Msg']}")
            label = "\\n".join(label_parts)

            # Prefer explicit Next: if present
            if "next" in info and info["next"] is not None:
                
                add_edge_safe(num, info["next"], label)
            
            else:
                # fallback to sequential
                next_step = all_step_nums[idx + 1] if idx + 1 < len(all_step_nums) else None
                if next_step:
                    
                    add_edge_safe(num, next_step, label)

        elif info["type"] == "StartState":
            next_step = all_step_nums[idx + 1] if idx + 1 < len(all_step_nums) else None
            if next_step:
                label = info.get("OutgoingLabel", info["label"])
               
                add_edge_safe(num, next_step, label)

        elif info["type"] == "DoState":
            if info["branches"]:
                for branch in info["branches"]:
                    target_step = branch["step"]
                    target_info = steps.get(target_step)
                    if target_info and target_info["type"] == "GotoStep" and target_info.get("goto"):
                        real_target = target_info["goto"]
                        
                        add_edge_safe(num, real_target, branch["desc"])
                    else:
                        
                        add_edge_safe(num, target_step, branch["desc"])
            else:
                next_step = all_step_nums[idx + 1] if idx + 1 < len(all_step_nums) else None
                if next_step:
                    label = info.get("OutgoingLabel", info["label"])
                    
                    add_edge_safe(num, next_step, label)

        if info["goto"]:
           
            add_edge_safe(num, info["goto"])
            continue
        
        else:
            next_step = all_step_nums[idx + 1] if idx + 1 < len(all_step_nums) else None
            has_outgoing = (
                bool(info["choices"]) or
                bool(info["branches"]) or
                bool(info["goto"]) or
                info["type"] in {"SendState", "DoState", "StartState"}
            )

            if next_step and not has_outgoing and info["type"] not in {"ReceiveState"}:
                
                add_edge_safe(num, next_step, "(default)")

    # Save final graph
    dot.render(output_path, cleanup=True)
    print(f"SBD graph saved at {output_path}.pdf")

In [15]:
def draw_all_subjects(sbd_text, output_folder):
    """
    Parses multi-subject SBD text, draws a graph per subject, and saves PDFs
    in output_folder with filenames as subject names.
    """
    print("Inside draw all subjects")
    subjects = parse_sbd_(sbd_text)
    print("Subjects found:", list(subjects.keys()))
    print("subjects: ",subjects)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for subject, subject_sbd_text in subjects.items():
        safe_subject = re.sub(r'[^\w\-_. ]', '_', subject).replace(' ', '_')
        output_path = os.path.join(output_folder, safe_subject) 
       
        draw_sbd_graph(subject_sbd_text, output_path)

In [16]:
from rdflib import Graph, Namespace, Literal  
from rdflib.namespace import RDF, RDFS, OWL, XSD
import re
from textwrap import dedent
from typing import Optional
import unicodedata

# Namespaces
ABSTRACT = Namespace("http://www.imi.kit.edu/abstract-pass-ont#")
STANDARD = Namespace("http://www.i2pm.net/standard-pass-ont#")
BASE     = Namespace("http://subjective-me.jimdo.com/s-bpm/processmodels/2025-03-25/Page-1#")

def normalize_name(name: str) -> str:
    import unicodedata
    name = unicodedata.normalize("NFKC", name)
    name = re.sub(r'\s+', ' ', name.strip().lower().rstrip(':'))
    name = name.replace("–", "-").replace("—", "-")
    return name

def sid_to_pass_owl(llama_text: str,
                    model_label: str = "PASS_Model",
                    out_file: Optional[str] = None) -> str:

    # STEP 1: Parse SID lines
    subjects = set()
    sid_lines = []
    sid_mode = False
    for ln in llama_text.splitlines():
        ln = ln.rstrip()
        print(ln)
        
        if re.search(r"###\s*(SID|Subject Interaction Diagram)", ln, re.IGNORECASE):
            sid_mode = True
            print("side_mode")
            continue
            
        if sid_mode and re.match(r"###\s+\w+", ln) and "SID" not in ln and "Subject Interaction Diagram" not in ln:
            break
            
        
                
        if sid_mode:
            ln_stripped = ln.lstrip()  # remove leading spaces
            if re.match(r"\d+\.", ln_stripped):
                sid_lines.append(ln_stripped)
                print(sid_lines)
                m = re.match(r"\d+\.\s*(.+?)\s*->\s*(.+?):", ln_stripped)
                if m:
                    sender, receiver = m.groups()
                    subjects.add(sender.strip())
                    subjects.add(receiver.strip())
                
    subjects = list(subjects)  
    print("SID lines:", sid_lines)
    print("Extracted subjects:", subjects)
    
    # STEP 2: Build RDF graph
    g = Graph(base=BASE)
    g.bind("abstract-pass-ont", ABSTRACT)
    g.bind("standard-pass-ont", STANDARD)
    g.bind("owl", OWL)
    g.bind("rdfs", RDFS)
    g.bind("xsd", XSD)

    model_uri = BASE[model_label]
    g.add((model_uri, RDF.type, STANDARD.PASSProcessModel))
    g.add((model_uri, STANDARD.hasModelComponentID, Literal(f"{model_uri}#Model", datatype=XSD.string)))
    g.add((model_uri, STANDARD.hasModelComponentLabel, Literal(model_label, lang="en")))

    sid_layer = BASE["SID_1"]
    g.add((sid_layer, RDF.type, ABSTRACT.ModelLayer))
    g.add((sid_layer, STANDARD.hasModelComponentID, Literal("SID_1", datatype=XSD.string)))
    g.add((sid_layer, STANDARD.hasModelComponentLabel, Literal("SID_1", lang="en")))
    g.add((sid_layer, STANDARD.hasPriorityNumber, Literal(1, datatype=XSD.positiveInteger)))
    g.add((model_uri, STANDARD.contains, sid_layer))

    # STEP 3: Subjects
    subj_id_map = {}
    for idx, subj_label in enumerate(subjects, start=2):
        sid = f"SID_1_FullySpecifiedSubject_{idx}"
        subj_uri = BASE[sid]
        subj_id_map[normalize_name(subj_label)] = subj_uri  # store normalized
        print(subj_id_map[normalize_name(subj_label)])

        g.add((subj_uri, RDF.type, STANDARD.FullySpecifiedSubject))
        g.add((subj_uri, STANDARD.hasModelComponentID, Literal(sid, datatype=XSD.string)))
        g.add((subj_uri, STANDARD.hasModelComponentLabel, Literal(subj_label, lang="en")))
        g.add((subj_uri, STANDARD.hasMaximumSubjectInstanceRestriction, Literal(1, datatype=XSD.integer)))
        g.add((subj_uri, ABSTRACT.hasExecutionCostPerHour, Literal(0.0, datatype=XSD.double)))

        g.add((sid_layer, STANDARD.contains, subj_uri))
        g.add((model_uri, STANDARD.contains, subj_uri))

    # STEP 4: Process SID messages
    mel_counter = 1
    msg_counter = 1
    for line in sid_lines:
        m = re.match(r"\d+\.\s*(.+?)\s*->\s*(.+?):\s*(.+)", line)
        if not m:
            continue
        sender, receiver, msg = m.groups()
        sender_uri   = subj_id_map[normalize_name(sender)]
        receiver_uri = subj_id_map[normalize_name(receiver)]

        msg_spec_id = f"SID_1_MessageSpecification_{msg_counter}"
        msg_spec_uri = BASE[msg_spec_id]
        g.add((msg_spec_uri, RDF.type, STANDARD.MessageSpecification))
        g.add((msg_spec_uri, STANDARD.hasModelComponentID, Literal(msg_spec_id, datatype=XSD.string)))
        g.add((msg_spec_uri, STANDARD.hasModelComponentLabel, Literal(msg, lang="en")))

        payload_id = f"PayloadDefinition_of_{msg_spec_id}"
        payload_uri = BASE[payload_id]
        g.add((payload_uri, RDF.type, OWL.Class))
        g.add((msg_spec_uri, STANDARD.containsPayloadDescription, payload_uri))

        mel_id = f"MessageExchangeList_on_SID_1_StandardMessageConnector_{mel_counter}"
        mel_uri = BASE[mel_id]
        conn_id = f"SID_1_StandardMessageConnector_{mel_counter}"
        conn_uri = BASE[conn_id]

        g.add((mel_uri, RDF.type, STANDARD.MessageExchangeList))
        g.add((mel_uri, STANDARD.hasModelComponentID, Literal(mel_id, datatype=XSD.string)))
        g.add((mel_uri, STANDARD.hasModelComponentLabel, Literal(conn_id, lang="en")))
        g.add((mel_uri, STANDARD.contains, msg_spec_uri))

        g.add((conn_uri, RDF.type, STANDARD.StandardMessageConnector))
        g.add((conn_uri, STANDARD.hasSender, sender_uri))
        g.add((conn_uri, STANDARD.hasReceiver, receiver_uri))
        g.add((conn_uri, STANDARD.hasMessageType, msg_spec_uri))
        g.add((mel_uri, STANDARD.contains, conn_uri))

        for parent in (sid_layer, model_uri):
            g.add((parent, STANDARD.contains, mel_uri))
            g.add((parent, STANDARD.contains, msg_spec_uri))
            g.add((parent, STANDARD.contains, conn_uri))

        mel_counter += 1
        msg_counter += 1  
    
    # === Detect ### SBD section first ===
    # STEP 5: Parse SBD section
    sbd_text_lines = []
    in_sbd_section = False
    for ln in llama_text.splitlines():
        ln_strip = ln.strip()
        if ln_strip.startswith("### SBD") or ln_strip.startswith("### Subject Behavior Diagram"):
            in_sbd_section = True
            continue
        if in_sbd_section and ln_strip.startswith("### "):
            break
        if in_sbd_section:
            sbd_text_lines.append(ln)

    sbd_sections = {}
    current_subject = None
    current_lines = []

    for ln in sbd_text_lines:
        ln_strip = ln.strip()
        if ln_strip.startswith("#### "):
            if current_subject and current_lines:
                sbd_sections[current_subject] = current_lines
            current_subject = ln_strip.replace("####", "").replace(":", "").strip()
            current_lines = []
        elif current_subject:
            current_lines.append(ln)
    if current_subject and current_lines:
        sbd_sections[current_subject] = current_lines


    # STEP 6: LOOP through EACH SBD section
    sbd_index = 1
    for sbd_name, sbd_lines in sbd_sections.items():
        subj_uri = subj_id_map.get(normalize_name(sbd_name))
        if not subj_uri:
            print(f"Warning: No matching FullySpecifiedSubject for {sbd_name}, skipping SBD")
            continue

        subj_sid_idx = subj_uri.split("_")[-1]
        sbd_id = f"SBD_{sbd_index}_SID_1_FullySpecifiedSubject_{subj_sid_idx}"
        sbd_uri = BASE[sbd_id]
        g.add((sbd_uri, RDF.type, STANDARD.SubjectBehavior))
        g.add((sbd_uri, STANDARD.hasModelComponentID, Literal(sbd_id, datatype=XSD.string)))
        g.add((sbd_uri, STANDARD.hasModelComponentLabel, Literal(f"SBD: {sbd_name}", lang="en")))
        g.add((sbd_uri, STANDARD.hasPriorityNumber, Literal(sbd_index, datatype=XSD.positiveInteger)))
        g.add((subj_uri, STANDARD.hasBehavior, sbd_uri))

        # --- Parse numbered states ---
        sbd_blocks = []
        current_block = []
        for ln in sbd_lines:
            if re.match(r"\d+\.", ln.strip()):
                if current_block:
                    sbd_blocks.append(current_block)
                current_block = [ln.strip()]
            else:
                if current_block:
                    current_block.append(ln.strip())
        if current_block:
            sbd_blocks.append(current_block)

        # --- Build lookup ---
        state_lookup = {}
        connections = []

        for block in sbd_blocks:
            m = re.match(r"(\d+)\.\s*(\w+State|GotoStep):\s*(.*)", block[0])
            if not m:
                continue
            step_num, state_type, label = m.groups()
            step_num = int(step_num)

            desc_val, to_val, from_val, msg_val = None, None, None, None
            next_steps = []
            branches = []
            choices = []
            goto_target = None

            in_choices = False
            in_branches = False

            for line in block[1:]:
                if m_desc := re.match(r"Description:\s*(.*)", line):
                    desc_val = m_desc.group(1).strip()
                if m_to := re.match(r"To:\s*(.*)", line):
                    to_val = m_to.group(1).strip()
                if m_from := re.match(r"From:\s*(.*)", line):
                    from_val = m_from.group(1).strip()
                if m_msg := re.match(r"Msg:\s*(.*)", line):
                    msg_val = m_msg.group(1).strip()
                if m_next := re.match(r"Next:\s*(\d+)", line):
                    next_steps.append(int(m_next.group(1)))
                if m_goto := re.match(r"GotoStep:\s*(\d+)", line):
                    goto_target = int(m_goto.group(1))

                # Choices start
                if line.startswith("Choices:"):
                    in_choices = True
                    continue
                if in_choices and line.startswith("Branches:"):
                    in_choices = False
                if in_choices and line.strip().startswith("-"):
                    # - From: ...  (followed by Msg & Next)
                    choice_from, choice_msg, choice_next = None, None, None
                    if m_from2 := re.match(r"-\s*From:\s*(.*)", line):
                        choice_from = m_from2.group(1).strip()
                    elif m_msg2 := re.match(r"Msg:\s*(.*)", line):
                        choice_msg = m_msg2.group(1).strip()
                    elif m_next2 := re.match(r"Next:\s*(\d+)", line):
                        choice_next = int(m_next2.group(1))
                        choices.append((choice_from, choice_msg, choice_next))

                # Branches start
                if line.startswith("Branches:"):
                    in_branches = True
                    continue
                if in_branches and line.strip().startswith("-"):
                    if m_step := re.match(r"-\s*Step:\s*(\d+)", line):
                        branches.append(int(m_step.group(1)))

            state_lookup[step_num] = {
                "type": state_type,
                "label": label,
                "desc": desc_val,
                "to": to_val,
                "from": from_val,
                "msg": msg_val
            }

            for n in next_steps:
                connections.append((step_num, n, None, None))
            for b in branches:
                connections.append((step_num, b, None, None))
            for cf, cm, cn in choices:
                connections.append((step_num, cn, cf, cm))
            if goto_target:
                connections.append((step_num, goto_target, None, None))
                

            # --- Create the state individual ---
            state_id = f"SBD_{sbd_index}_{state_type}_{step_num}"
            state_uri = BASE[state_id]
            g.add((state_uri, RDF.type, STANDARD[state_type]))
            g.add((state_uri, STANDARD.hasModelComponentID, Literal(state_id, datatype=XSD.string)))
            g.add((state_uri, STANDARD.hasModelComponentLabel, Literal(label, lang="en")))
            g.add((sbd_uri, STANDARD.contains, state_uri))

        # --- Create transitions ---
        transition_counter = 1
        for source_step, target_step, choice_from, choice_msg in connections:
            src_info = state_lookup[source_step]
            tgt_info = state_lookup[target_step]

            trans_id = f"SBD_{sbd_index}_{src_info['type']}Transition_{transition_counter}"
            trans_uri = BASE[trans_id]

            if src_info['type'] == "SendState":
                trans_type = STANDARD.SendTransition
                trans_label = f"To: {src_info['to']}\nMsg: {src_info['msg']}"
            elif src_info['type'] == "ReceiveState":
                trans_type = STANDARD.ReceiveTransition
                if choice_from or choice_msg:
                    trans_label = f"From: {choice_from}\nMsg: {choice_msg}"
                else:
                    trans_label = f"From: {src_info['from']}\nMsg: {src_info['msg']}"
            elif src_info['type'] == "GotoStep":
                trans_type = STANDARD.DoTransition
                trans_label = f"Go to step {target_step}"
            else:
                trans_type = STANDARD.DoTransition
                trans_label = src_info['desc'] or "Continue Process"

            g.add((trans_uri, RDF.type, trans_type))
            g.add((trans_uri, STANDARD.hasModelComponentID, Literal(trans_id, datatype=XSD.string)))
            g.add((trans_uri, STANDARD.hasModelComponentLabel, Literal(trans_label, lang="en")))
            g.add((trans_uri, STANDARD.hasSourceState, BASE[f"SBD_{sbd_index}_{src_info['type']}_{source_step}"]))
            g.add((trans_uri, STANDARD.hasTargetState, BASE[f"SBD_{sbd_index}_{tgt_info['type']}_{target_step}"]))
            g.add((sbd_uri, STANDARD.contains, trans_uri))
            transition_counter += 1

        g.add((sid_layer, STANDARD.contains, sbd_uri))
        g.add((model_uri, STANDARD.contains, sbd_uri))
        sbd_index += 1
        
    # example class
    dm_class = BASE["VisioShapesInternalDataMappingFunction"]
    g.add((dm_class, RDF.type, OWL.Class))
    g.add((dm_class, RDFS.subClassOf, STANDARD.DataMappingFunction))

    # STEP 7: Serialize RDF/XML
    xml_body = g.serialize(format="application/rdf+xml")

    entities = dedent("""\
        <!DOCTYPE rdf:RDF [
            <!ENTITY owl "http://www.w3.org/2002/07/owl#" >
            <!ENTITY xsd "http://www.w3.org/2001/XMLSchema#" >
            <!ENTITY rdfs "http://www.w3.org/2000/01/rdf-schema#" >
            <!ENTITY abstract-pass-ont "http://www.imi.kit.edu/abstract-pass-ont#" >
            <!ENTITY standard-pass-ont "http://www.i2pm.net/standard-pass-ont#" >
            <!ENTITY rdf "http://www.w3.org/1999/02/22-rdf-syntax-ns#" >
        ]>
    """)
    xml_body_nohead = "\n".join(xml_body.splitlines()[1:])
    final_xml = f'<?xml version="1.0"?>\n{entities}\n{xml_body_nohead}'

    if out_file:
        with open(out_file, "w", encoding="utf-8") as f:
            f.write(final_xml)

    return final_xml

In [17]:
import panel as pn
from openai import OpenAI
import re
import panel as pn
from PIL import Image
from io import BytesIO

# Add a Panel pane to display the image
image_view = pn.pane.PNG(width=700)

pn.extension()

# --- API setup ---
api_key = ""
base_url = "https://gpt.uni-muenster.de/v1"
model = "Llama-3.3-70B"  # model used
client = OpenAI(api_key=api_key, base_url=base_url)

# --- One-shot example prompt ---
one_shot_example = """
You are an expert in Subject-Oriented Business Process Modeling using the Parallel Activity Specification Schema (PASS).

Given the following scenario:

\"\"\"{scenario}\"\"\"


Your task is to generate a PASS model in two parts:

### 1. Subject Interaction Diagram (SID):

- Identify all the **subjects** involved.
-Step 1: List all subjects explicitly.
Before generating any interactions, provide a numbered list of all distinct subjects involved in the scenario.
Use only their names (e.g., Customer, Supplier, Warehouse).
- Describe their **interactions** using messages in **noun form only** (e.g., "Order", "Inventory Check Request", "Confirmation Email").
- Avoid using verbs like “place”, “send”, “check”, or “confirm”.
- Use noun phrases that represent the content of the interaction, not the action.
- Return the SID as a numbered list of interactions using the format:
  Subject A -> Subject B: Message (noun form)
  
### 2. Subject Behavior Diagram (SBD):

### Rules for the SBD:

- For each subject, describe their **internal behavior** as a sequence of states.
- Use the following state types and label each step explicitly:
  - **StartState**: the beginning of the behavior
  - **ReceiveState**: receiving a message
  - **SendState**: sending a message
      - For every **SendState**, include the following two fields on the next lines:
      - `To:` — the name of the recipient subject
      - `Msg:` — the message being sent, written as a **noun or noun phrase** (e.g., `Order`, `Payment`, `Order Cancellation Request`)

      Example:
      2. SendState: Send Order to E-commerce Website  
         To: E-commerce Website  
         Msg: Order

- **DoState**: internal processing or logic
  - **EndState**: conclusion of the subject's behavior
  
- Ensure each interaction in the SID maps to a SendState and ReceiveState in the corresponding SBDs.
- For every **ReceiveState**, also include the `From:` (sender subject) and `Msg:` (the received noun message) on the next line below it.
  - The `Msg:` must be a **noun or noun phrase only**, matching the noun form used in the SID (e.g., "Order", "Appointment Confirmation").  
  - Do **not** use verbs, verb phrases, or action words (e.g., "send payment", "check availability").

  
### Control Flow & Branching Logic (Important):
- A `DoState` can act as a **decision point**.
- If a state has multiple `Branches:`, they represent **mutually exclusive outcomes**, not parallel or sequential steps.
- Use `Branches:` to list possible next steps in decision states.
- To represent non-linear flow, use `GotoStep:` to explicitly jump back or forward to a specific step.
- Do **not** assume that steps are sequential by default—follow the above rules.
- End a branch with `EndState` or no further transitions to mark it as terminal.

-Use GotoStep: to define non-linear transitions (e.g., retry loops or skipping steps).
-A ReceiveState can lead to multiple different outcomes based on the received message.
   Use a Choices: block to specify alternative messages and the corresponding next steps.
   Each choice must include:
     From: — the sender subject
     Msg: — the received message (noun phrase)
     Next: — the step that follows if that message is received
     
Clarifications for ReceiveState and Control Flow:

- A `ReceiveState` with `Choices:` should **not** continue into the next numeric step by default.
- The `Next:` field under each `Choice:` is the **only** path to follow.
- Ensure each `Next:` maps to a valid step with correct logic. Do **not** assume the next numeric step follows sequentially.
- If a message like `Order Declination Message` is received, it should go **directly to the correct decision step** (e.g., Decide upon further action), not to a step meant for confirmations (e.g., Step 5).
- Use `GotoStep:` to connect flows clearly after decisions or retries.
- Branches in a `DoState` represent **mutually exclusive decisions** and should not be followed sequentially.

### Additional Structural Rules for Valid ReceiveStates:

- Each `ReceiveState` should group only messages that are logically possible **at that specific point in the behavior**. Do not mix messages from different process phases or inconsistent branches.
  
- After a subject has performed a critical action (e.g., sent a payment, confirmed a booking), they should not receive a message that contradicts or invalidates that action (e.g., an order declination after payment).  
  - Such contradictory messages should be handled **before** that action was taken — not afterward.

- A `ReceiveState` must not mix messages from **different sender subjects** that belong to **separate branches of logic**. For example:
  - If one message implies success from one subject (e.g., confirmation email from System A)
  - And another implies failure from another subject (e.g., declination from System B)
  - These should not be in the same `Choices:` block. They should belong to **different ReceiveStates**, based on earlier branching.

- Always ensure that once a decision or branching has occurred, only the resulting possible messages **from that path** are expected in the next `ReceiveState`.

- If a message like `Order Declination` has already been handled in a previous `ReceiveState` as a decision point, it should **not appear again** in later states.

### Additional Rule: Include Description for StartState and DoState

- For every **StartState** and **DoState**, provide a `Description:` line immediately below the step.
- This line must concisely describe what causes the state to exit or what condition/decision leads to the next transition.
- This helps label the outgoing edge in visual diagrams.

**Examples**:

1. StartState: Decide to place an order  
   Description: Customer initiates order process

2.DoState: Pick Order
  Description: Order picked

### Structural and Control Flow Guidance:

- Do **not** create redundant ReceiveState steps. If a message is received at multiple points (e.g., "Order Declination Message"), handle all logic in the **same ReceiveState step**, using `Choices:` and `Next:`.
  - Do **not** repeat the same ReceiveState (e.g., ReceiveState: Receive Reply from E-commerce Website) if some previous Step already handles the same messages.
- If a behavior is a repeat (e.g., re-sending the order), do **not** create a new `SendState`.  
  - Use `GotoStep:` to return to the original step (e.g., `GotoStep: 2`), rather than creating a redundant step like "SendState: Send Order for Another Supplier to E-commerce Website".

- `GotoStep:` should be used for loops (e.g., retrying with another supplier). Avoid creating a new SendState when the action already exists in a previous step.

- When listing `Branches:`, each branch should either:
  - Lead to an `EndState`, or
  - Lead to another state (e.g., `DoState`, `SendState`) and eventually terminate or loop.
  
###IMPORTANT:  
-If the ReceiveState choice is regarding Declination or rejection then next state number should belong to Decide upon further action. 
  
### Additional Rules for Correct EndStates and Branching:

- When using a `DoState` with `Branches:`, each branch must:
  - Lead clearly to an `EndState`, OR
  - Use a `GotoStep:` to continue the flow elsewhere.

- If a branch leads to an intermediate action (like sending a cancellation request), then that action **must be followed explicitly by an `EndState`** unless the flow continues elsewhere.

- Do **not** include orphan `EndState` steps that are never referenced via `Next:` or `GotoStep:`. Every `EndState` must be clearly reachable in the flow.

- You may combine a final action and end state if there is no further processing (e.g., "Send Cancellation Request and terminate" can be modeled in a single `SendState` followed immediately by `EndState`).

- When retrying an earlier action (like sending a new order), use `GotoStep:` to return to that earlier state. Do not re-declare the same SendState.

### Final State Rule:
- A single step cannot combine a `SendState` and `EndState`.  
- If a final action like sending a cancellation request ends the process:
  - Use two steps:
    - One `SendState`
    - One `EndState` directly after
- Example:
  9. SendState: Send Cancellation Request to E-commerce Website  
  10. EndState: Order Cancelled

- Then reference Step 10 explicitly from any branch or choice that leads there.

###ABSOLUTE Next: RULE (HIGHEST PRIORITY):
-Every state must explicitly include a Next: line. No exceptions.

-Next: must always appear immediately under each state (even when sequential).

-Next: must point to the next valid step number in the flow.

For example:-
3. SendState: Send Order to Supplier
   To: Supplier
   Msg: Order
   Next: 4

4. ReceiveState: Receive Order Confirmation from Supplier
   From: Supplier
   Msg: Order Confirmation
   Next: 5


### Post-Payment Behavior(STRICT RULE):

- After sending a payment, the Customer must wait for a response using a `ReceiveState`.
- This `ReceiveState` should include:
  - A `Payment Confirmation` message that proceeds to an `EndState` or confirmation step.
  - A `Payment Declination` message that leads to a decision point (`DoState`).

-You CANNOT:
 -Jump directly from Send Payment to DoState
 -Combine sending a payment and deciding what to do in the same step

### Avoid Ambiguous Transitions:

- Do not place a `DoState` directly after sending a payment.
- A `DoState` is only appropriate after:
  - A declination has been received.
  - The customer must choose between canceling or retrying.

### Retry Logic:

- If a retry loop is needed, use:
  - `GotoStep:` to return to the appropriate order initiation step.
- Ensure all retry paths explicitly flow to `GotoStep`, not a dangling step.

### IMPORTANT Branch Semantics and Retry Detection Rules:
-If a Branch has a description that includes any of the following retry-like phrases:

 -choose another [supplier|hospital|partner]
 -try again
 -retry
 -resend
 -submit again
 -place another order

Then that branch must lead to a GotoStep pointing to the step where the original send action occurred (e.g., sending the order or request).
Do not create a new SendState — reuse the earlier one via GotoStep:.
This enables looping retry logic without duplication.
These kinds of retry branches should never point to an EndState.
If the intent of the branch is to cancel or stop, use a SendState for final communication and then an EndState.

Example:
5. DoState: Decide upon further action
   Branches:
     - Step: 6  
       Description: Cancel order
     - Step: 8
       Description: Choose another supplier
6. SendState: Send Order Cancellation Request
7. EndState: Order Cancelled
8. GotoStep: 2  # Retry order loop

### Step Types:

- A step must use **only one state type**: either `SendState`, `ReceiveState`, `DoState`, or `EndState`.
- Do not combine `SendState` and `EndState` in one step.


- If a retry is needed (e.g., choosing another supplier), use a `GotoStep:` to return to the appropriate earlier step (usually where the order is sent).
- Do **not** define a `GotoStep:` without referencing it in any branch.
- Every GotoStep must be **actually used in a `Branches:` section** of a `DoState`.

- Every `GotoStep:` must be referenced explicitly in a decision branch or as a next step in the control flow.
- Do not create `GotoStep:` steps that are not reachable from any previous state.
- When defining `Branches:` in a `DoState`, ensure that each branch points to a valid next step, including `GotoStep:` steps if needed.
- For example, if you define a retry loop as `GotoStep: 2`, ensure the decision branch description points to that step number.
- Avoid orphan steps that are unreachable from the flow. Like this below, 8. GotoStep: 2  # Retry appointment loop is referenced explicitly
in Step 5 - Step: 8
       Description: Choose another hospital and try again :

5. DoState: Decide upon further action
   Branches:
     - Step: 6  
       Description: Cancel appointment request 
     - Step: 8
       Description: Choose another hospital and try again  
6. DoState: Cancel appointment request
7  EndState: Appointment request cancelled (end of this path)
8. GotoStep: 2  # Retry appointment loop

##IMPORTANT: Global Unique Step Numbering Rule

-Each line in the SBD must have a globally unique step number.

-Never reuse a number for two different steps, even across branches.

-Numbering must always increase sequentially, without gaps or duplicates.


###ABSOLUTE ENDSTATE RULE
-Every branch of the behavior must explicitly terminate in a unique EndState.

-A path that leads to cancellation must end in EndState: Order cancelled.

-A path that leads to successful fulfillment must end in EndState: Order confirmed and paid (or the correct success wording).

-No branch can skip its own EndState or jump to an unrelated EndState.

-Retry loops are the only branches that use GotoStep: and do not end. But if the user finally cancels or succeeds, you must always add a proper EndState.

-Do NOT leave a SendState as the last step without an EndState.

-Do NOT make a cancellation branch jump to a success EndState, and vice versa.

  
### Use the following format (example based on a healthcare scenario):

### Example Scenario:
A patient books an appointment using a healthcare app.  
The app sends the appointment request to the hospital system.  
The hospital system checks the doctor’s availability.  
If the doctor is available, it confirms the appointment.  
The app then sends a confirmation message to the patient.
If the doctor is not available, hospital system declines the appointment.
The patient is sent decline message.
The patient then decides upon further action either the patient cancels request and all ends
or the patient chooses another hospital.

### SBD:

#### Patient:
1. StartState: Decide to make appointment
   Description: Patient initiates appointment process
2. SendState: Send Appointment Request to Healthcare App  
   -To:  Healthcare App
   -Msg: Appointment Request
3. ReceiveState: Receive Reply from Healthcare App
   Choices:
   -From: Healthcare App  
    Msg: Appointment Confirmation   
    Next: 4
   -From: Healthcare App
    Msg: Appointment Declined
    Next: 5 
   
4. EndState: Appointment booked
5. DoState: Decide upon further action
   Branches:
     - Step: 6  
       Description: Cancel appointment request 
     - Step: 8
       Description: Choose another hospital and try again  
6. DoState: Cancel appointment request
   Description: Cancel
7  EndState: Appointment request cancelled (end of this path)
8. GotoStep: 2  # Retry appointment loop

Note: Step 5 must be the exclusive next step after receiving an "Appointment Declined" message. Similarly, in the e-commerce context, if the customer receives an "Order Declination Message", it must always go to this decision point, never to a step meant for payment.

## IMPORTANT RULE:
If the `Customer` receives an `Declination Message`, it must transition to a `DoState` titled `Decide upon further action`. This state must contain:
- A branch to cancel the order (leading to a SendState, then EndState)
- A branch to choose another supplier (leading to `GotoStep: 2`)

Under no condition should the `Order Declination Message` lead directly to the payment step or any unrelated state.

##IMPORTANT:
Ensure that state numbering reflects logical flow.
For example:

If a payment is sent in Step 4, then receiving the payment response must occur immediately after (e.g., in Step 5 or Step 5.x), not at the end.

Avoid placing follow-up states (like payment confirmation) far from their corresponding initiator steps.

Only GotoStep or loops should cause jumps in the flow.

Try to keep closely related actions (send/receive pairs, decision/action results) in proximity and numbered accordingly.


##Cancellation Branch Sequencing Rule:
If a branch description includes "Cancel order" (or any cancellation intent), it must:

Point to the SendState step that sends the cancellation request (e.g., Step 9).

That SendState’s Next: must point to the EndState for cancellation (e.g., Step 10).

Never point directly from the branch to the EndState.


##No Orphan Step Rule:
Every step number must be explicitly referenced in a Next: field, GotoStep:, or Branches: section of a previous step.
Steps that are never reached from any earlier step are not allowed.
If a message is already handled in a previous ReceiveState (e.g., Order Confirmation Email in Step 5), do not create a new step for receiving it later.
All ReceiveState steps must be placed only where they are actually reached in the control flow.
You must check that each step is connected to at least one incoming reference; otherwise, remove or merge it into the relevant existing step.


Now, generate the SBD for all subjects.

Important Correction:
If the customer chooses to try another supplier, use `GotoStep: 2` (Send Order....) rather than `GotoStep: 3` (Receive Response). The retry loop must start from the original `SendState`, not from the `ReceiveState`.
"""

# --- Panel Widgets ---
scenario_input = pn.widgets.TextAreaInput(
    name='PASS Scenario',
    value='''''',
    height=150,
    width=700
)

generate_btn = pn.widgets.Button(name="Generate SID and SBD", button_type='primary')



output_markdown = pn.pane.Markdown(
  
    sizing_mode="stretch_width"
)

output_area = pn.Column(
    output_markdown,
    max_height=400,
    sizing_mode="stretch_width",
    styles={"overflow": "auto"}
)

# --- Button callback ---
def generate_sid(event):
    output_area.object = ""
    image_view.object = None
    scenario = scenario_input.value.strip()
    if not scenario:
        output_area.object = "Please enter a scenario description."
        return

    prompt = one_shot_example.replace("{scenario}", scenario)

    try:
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=model,
            temperature=0.5
            
        )
        sid_text = response.choices[0].message.content.strip()
        
       
        print("sid_text:",sid_text)
        
        output_markdown.object = f"### Identified Subjects and Interactions:\n\n```\n{sid_text}\n```"


        # --- Graph Drawing ---
        sid_pairs = parse_sid(sid_text)
        print("sid_pairs:",sid_pairs)
        process = {
                    "SID": sid_pairs
                  }
       
        from datetime import datetime
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        graph_path = fr"E:\Thesis\PASS Diagrams\Diagram\llama\sid_graph-{timestamp}"
        print("Graph saved at:", graph_path + ".png")
       
        draw_sid_graph(
                       process["SID"],
                       output_path=graph_path
                      )
        
        import os
        print("Exists:", os.path.exists(graph_path + ".png"))


       
        img = Image.open(graph_path + ".png")
        img.show()
                
        
        
        sbd_textt = extract_sbd_section(sid_text)
        print
        
       
        
        if sbd_textt:
            print(sbd_textt)
            output_folder = fr"E:\Thesis\PASS Diagrams\Diagram\llama"
            draw_all_subjects(sbd_textt,output_folder)
            
        else:
            print("No SBD section found in response!")
            
        xml = sid_to_pass_owl(sid_text, out_file="E:\Thesis\PASS Diagrams\OWL_File\OWLFile-Llama.owl")

    except Exception as e:
        output_area.object = f"API Error: {e}"
        image_view.object = None

generate_btn.on_click(generate_sid)


results_panel = pn.Column(
    output_area,
    pn.Spacer(height=20),
   
    image_view,
    sizing_mode="stretch_width"
)

app = pn.Column(
    pn.pane.Markdown("# PASS Process Modeling with LLM"),
    scenario_input,
    generate_btn,
    results_panel,
    sizing_mode="stretch_width"
)




scrollable_app = pn.Column(
    app,
    height=800,
    sizing_mode="stretch_width",
    styles={"overflow-y": "auto"}
)
scrollable_app.show()

Launching server at http://localhost:61219


sid_text: ### Subject Interaction Diagram (SID):

1. Customer -> E-commerce Website: Order
2. E-commerce Website -> Inventory System: Inventory Check Request
3. Inventory System -> E-commerce Website: Inventory Check Response
4. E-commerce Website -> Customer: Order Confirmation Request
5. Customer -> E-commerce Website: Payment
6. Payment Gateway -> E-commerce Website: Payment Confirmation
7. E-commerce Website -> Customer: Order Confirmation Email
8. E-commerce Website -> Customer: Order Declination Message
9. Customer -> E-commerce Website: Order Cancellation Request
10. Customer -> Another E-commerce Supplier: Order

### Subject Behavior Diagram (SBD):

#### Customer:
1. StartState: Decide to place an order
   Description: Customer initiates order process
   Next: 2
2. SendState: Send Order to E-commerce Website
   To: E-commerce Website
   Msg: Order
   Next: 3
3. ReceiveState: Receive Response from E-commerce Website
   Choices:
   - From: E-commerce Website
     Msg: Order Confi

SBD graph saved at E:\Thesis\PASS Diagrams\Diagram\llama\Customer.pdf
SBD graph saved at E:\Thesis\PASS Diagrams\Diagram\llama\E-commerce_Website.pdf
SBD graph saved at E:\Thesis\PASS Diagrams\Diagram\llama\Inventory_System.pdf
SBD graph saved at E:\Thesis\PASS Diagrams\Diagram\llama\Payment_Gateway.pdf
SBD graph saved at E:\Thesis\PASS Diagrams\Diagram\llama\Another_E-commerce_Supplier.pdf
### Subject Interaction Diagram (SID):
side_mode

1. Customer -> E-commerce Website: Order
['1. Customer -> E-commerce Website: Order']
2. E-commerce Website -> Inventory System: Inventory Check Request
['1. Customer -> E-commerce Website: Order', '2. E-commerce Website -> Inventory System: Inventory Check Request']
3. Inventory System -> E-commerce Website: Inventory Check Response
['1. Customer -> E-commerce Website: Order', '2. E-commerce Website -> Inventory System: Inventory Check Request', '3. Inventory System -> E-commerce Website: Inventory Check Response']
4. E-commerce Website -> Customer: